In [1]:
import os

In [2]:
%pwd

'd:\\projects\\text summarizer NLP project\\Text-Summarizer-NLP-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\projects\\text summarizer NLP project\\Text-Summarizer-NLP-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        print(f"Loaded config: {self.config}")
        print(f"Loaded params: {self.params}")

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.get('TrainingArguments', {})

        # Provide default values if necessary
        num_train_epochs = params.get('num_train_epochs', 3)
        warmup_steps = params.get('warmup_steps', 500)
        per_device_train_batch_size = params.get('per_device_train_batch_size', 1)
        weight_decay = params.get('weight_decay', 0.01)
        logging_steps = params.get('logging_steps', 10)
        evaluation_strategy = params.get('evaluation_strategy', 'steps')
        eval_steps = params.get('eval_steps', 500)
        save_steps = params.get('save_steps', 1e6)
        gradient_accumulation_steps = params.get('gradient_accumulation_steps', 16)

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(

            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=num_train_epochs,
            warmup_steps=warmup_steps,
            per_device_train_batch_size=per_device_train_batch_size,
            weight_decay=weight_decay,
            logging_steps=logging_steps,
            evaluation_strategy=evaluation_strategy,
            eval_steps=eval_steps,
            save_steps=save_steps,
            gradient_accumulation_steps=gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\LENOVO\Anaconda3\envs\textS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\LENOVO\Anaconda3\envs\textS\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\LENOVO\A

[2024-06-27 20:31:33,154: INFO: config: PyTorch version 2.3.1 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps= 500,
            per_device_train_batch_size=1, per_device_eval_batch_size= 1,
            weight_decay= 0.01, logging_steps= 10,
            evaluation_strategy= 'steps', eval_steps= 500, save_steps= 1e6,
            gradient_accumulation_steps= 16  
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config= ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-27 20:31:33,693: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-27 20:31:33,697: INFO: common: yaml file: params.yaml loaded successfully]
Loaded config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/b-1129/Text-Summarizer-NLP-Project/raw/main/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}}
Loaded p

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\LENOVO\Anaconda3\envs\textS\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 20%|█▉        | 10/51 [46:53<3:52:20, 340.01s/it]

{'loss': 3.1101, 'grad_norm': 31.946226119995117, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 22%|██▏       | 11/51 [51:09<3:29:31, 314.30s/it]

KeyboardInterrupt: 